In [16]:
import json
#import _cvxcore
#import cvxpy
import fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,SimpleFill
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from argparse import ArgumentParser
import math

In [31]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--pred', default=False)
ARG_PARSER.add_argument('--imp', default=True)

ARG_PARSER.add_argument('--pred_len', default=5, type=int)
ARG_PARSER.add_argument('--missingRate', default=50, type=int)

_StoreAction(option_strings=['--imp'], dest='imp', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help=None, metavar=None)

In [32]:
def get_loss(X, X_pred, Y):
    # find ones in Y but not in X (ground truth)
    mask = np.isnan(X) ^ np.isnan(Y)

    X_pred = np.nan_to_num(X_pred)
    pred = X_pred[mask]
    label = Y[mask]

    mae = np.abs(pred - label).sum() / (1e-5 + np.sum(mask))
    mre = np.abs(pred - label).sum() / (1e-5 + np.sum(np.abs(label)))

    return {'mae': mae, 'mre': mre}

In [33]:
def dataPrep(data,flag):
    if flag==0:
        del data['person_id']
    #print(data['Age'].head())
    #print(data.columns)
    if flag==1:
        del data['interval']
        del data['intervalReverse']
        #print(data.head)
    #print(data.shape)
    data = data.to_numpy()
    data = np.reshape(data, (int(data.shape[0]/40), 40, data.shape[1]))
    #data=data[2500:3000,:,:]
    #print(data.shape)
    return data

In [34]:
args = ARG_PARSER.parse_args(args=[])

In [ ]:
def ehrPred(predWin):
    dataF=pd.read_csv('.../aaai/data/ehr/preprocess/test/condFtest.csv',header=0)
    maskF=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/'maskFtest.csv',header=0)
    dataM=pd.read_csv('.../aaai/data/ehr/preprocess/test/condMtest.csv',header=0)
    maskM=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/maskMtest.csv',header=0)
    
    data=pd.concat([dataF,dataM])
    mask=pd.concat([maskF,maskM])
    data['BMI']=mask['BMI']
    
    data=dataPrep(data,0)
    mask=dataPrep(mask,1)
    
    mask=mask[:,:,4]
    y=data[:,:,811].copy()
    testMask=mask.copy()
    
    #print(data.shape)
    #print(mask.shape)
    #print(y.shape)
    #print(testMask.shape)

    for i in range(data.shape[0]):
        #if(data[i,])
        j=40
        if(predWin==8):
            k=32
        elif(predWin==7):
            k=28
        elif(predWin==6):
            k=24
        elif(predWin==5):
            k=20

        data[i,j-k:j,:]=0
        mask[i,j-k:j]=0
        y[i,0:j-k]=0
        testMask[i,0:j-k]=0

    data=data[:,:,811]
    
    data[data==0]=np.nan
#     print("mean",data.stack().mean())
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())
    
#     data_complete=data_complete.transpose()
#     data=data.transpose()
#     mask=mask.transpose()
#     y=y.transpose()
#     testMask=testMask.transpose()
    
#     data_complete=data_complete.fillna(data_complete.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)

    #print(data_complete[0,:])
    #print(y[0,:])
    #print(testMask[0,:])
    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]
#     with open('results/mean/'+str(m) +'/outBmiImp', 'wb') as fp:
#         pickle.dump(outBmi, fp)
#     with open('results/mean/'+str(m) +'/inBmiImp', 'wb') as fp:
#         pickle.dump(inBmi, fp)
    #print(len(inBmi))
    #print(len(outBmi))
    #plotBmi(inBmi,outBmi,len(inBmi))

    print("Mean MAE: ",mae)
    print("Mean MRE: ",mre)

In [ ]:
def ehrImpute(missingRate):
    dataF=pd.read_csv('.../aaai/data/ehr/preprocess/test/condFtest.csv',header=0)
    maskF=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/'maskFtest.csv',header=0)
    dataM=pd.read_csv('.../aaai/data/ehr/preprocess/test/condMtest.csv',header=0)
    maskM=pd.read_csv('.../aaai/data/ehr/preprocess/test/mask/maskMtest.csv',header=0)
    
    data=pd.concat([dataF,dataM])
    mask=pd.concat([maskF,maskM])
    data['BMI']=mask['BMI']
    
    data=dataPrep(data,0)
    mask=dataPrep(mask,1)
    
    mask=mask[:,:,4]
    y=data[:,:,811].copy()
    testMask=mask.copy()
    
    #print(data.shape)
    #print(mask.shape)
    #print(y.shape)
    #print(testMask.shape)
    
    samples=0
    pids=0
    
    #print(mask.sum())

    #print(range(data.shape[0]))
    #print(data[100,:,811])
    #print(mask[100,:])
    for i in range(data.shape[0]):
        idxs = np.nonzero(mask[i,:] == 1)
        idxs=(np.array(idxs)).reshape(-1)
        #idxs=np.random.choice(idxs,3)
        #print(len(idxs))
        samples=samples+len(idxs)
        if((missingRate==50) & (len(idxs)>4)):
            idxs=np.random.choice(idxs,5,replace=False)
            data[i,idxs[0],811]=0
            data[i,idxs[1],811]=0
            data[i,idxs[2],811]=0
            data[i,idxs[3],811]=0
            data[i,idxs[4],811]=0
            #print(mask[i,:])
            mask[i,idxs[0]]=0
            mask[i,idxs[1]]=0
            mask[i,idxs[2]]=0
            mask[i,idxs[3]]=0
            mask[i,idxs[4]]=0
            pids=pids + 5
        elif((missingRate>=40) & (len(idxs)>3)):
            idxs=np.random.choice(idxs,4,replace=False)
            data[i,idxs[0],811]=0
            data[i,idxs[1],811]=0
            data[i,idxs[2],811]=0
            data[i,idxs[3],811]=0
            #print(mask[i,:])
            mask[i,idxs[0]]=0
            mask[i,idxs[1]]=0
            mask[i,idxs[2]]=0
            mask[i,idxs[3]]=0
            pids=pids + 4
        elif((missingRate>=30) & (len(idxs)>2)):
            idxs=np.random.choice(idxs,3,replace=False)
            data[i,idxs[0],811]=0
            data[i,idxs[1],811]=0
            data[i,idxs[2],811]=0
            #print(mask[i,:])
            mask[i,idxs[0]]=0
            mask[i,idxs[1]]=0
            mask[i,idxs[2]]=0
            pids=pids + 3
        elif((missingRate>=20) & (len(idxs)>1)):
            idxs=np.random.choice(idxs,2,replace=False)
            data[i,idxs[0],811]=0
            data[i,idxs[1],811]=0
            #print(mask[i,:])
            mask[i,idxs[0]]=0
            mask[i,idxs[1]]=0
            pids=pids + 2
        elif((missingRate>=10) & (len(idxs)>0)):
            if (i%2==0):
                idxs=np.random.choice(idxs,1,replace=False)
                data[i,idxs,811]=0
                mask[i,idxs]=0
                pids=pids + 1

        testMask[i,:]=testMask[i,:]-mask[i,:]
        y[i,:]=y[i,:]*testMask[i,:]
        
    print("samples",samples)
    print("pids",pids)
    print("missing%",pids/samples)
    data=data[:,:,811]
    data=data.transpose()
    mask=mask.transpose()
    y=y.transpose()
    testMask=testMask.transpose()
    data[data==0]=np.nan
    data=pd.DataFrame(data)

    data_complete=data.fillna(data.mean())

    data=data.fillna(0)
    data_complete=data_complete.fillna(0)

    data = data.to_numpy()
    data_complete = data_complete.to_numpy()

    data_complete=np.multiply(data_complete,testMask)

    
    #print(data_complete[100,:])
    #print(y[100,:])
    #print(testMask[100,:])
    mae = np.abs(data_complete - y).sum() / (1e-5 + np.sum(testMask))
    mre = np.abs(data_complete - y).sum() / (1e-5 + np.sum(np.abs(y)))

    outBmi=data_complete[np.nonzero(testMask)]
    inBmi=y[np.nonzero(testMask)]
#     with open('results/mean/'+str(m) +'/outBmiImp', 'wb') as fp:
#         pickle.dump(outBmi, fp)
#     with open('results/mean/'+str(m) +'/inBmiImp', 'wb') as fp:
#         pickle.dump(inBmi, fp)
    #print(len(inBmi))
    #print(len(outBmi))
    #plotBmi(inBmi,outBmi,len(inBmi))

    print("Mean MAE: ",mae)
    print("Mean MRE: ",mre)

In [ ]:
#MEAN PRED ALGO
if args.pred:
    ehrPred(8)
    ehrPred(7)
    ehrPred(6)
    ehrPred(5)

In [ ]:
#MEAN IMPUTATION ALGO
if args.imp:
    ehrImpute(10)
    ehrImpute(20)
    ehrImpute(30)
    ehrImpute(40)
    ehrImpute(50)